In [108]:
## importing all the neccessary package for this project
import numpy as np
import pandas as pd
import time
import datetime
from datetime import datetime, timezone
import requests
import json
import tweepy
import io
from io import BytesIO
import os
import re

<style>
    /* define the default link style */
    a {
        color: gray;
        text-decoration: none;
    }
    
    /* define the link style when it is clicked */
    a.clicked {
        color: blue;
    }
    
    /* define the link style when it is hovered */
    a:hover {
        color: black;
    }
</style>

<ul>
    <li><a href="#Gather"><strong><font size="+2">Gather</font></strong></a></li>
    <li><a href="#Assess"><strong><font size="+2">Assess</font></strong></a></li>
    <li><a href="#Clean"><strong><font size="+2">Clean</font></strong></a></li>
    <li><a href="#Store"><strong><font size="+2">Store</font></strong></a></li>
</ul>


<a id='Gather'></a>
# **Gather**
There are three data to collect for this analysis.

first is the "twitter_archive_enhanced" in CSV,

then, the "image_predictions" in TSV in URL,

lastly, the query the Twitter API and collect Data on retweet_count and like_count. (This has been provided by Udacity instructor since twitter has failed to provide me with an API access)

In [109]:
## load the twitter_archive_enhanced.csv file on df using pandas
filepath = "twitter-archive-enhanced-2 (2).csv"
twitter_archive_df = pd.read_csv(filepath)                         
twitter_archive_df.head(2)


,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Phineas. He's a mystical boy. Only ever appears in the hole of a donut. 13/10 https://t.co/MgUWQ76dJU,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643555336193/photo/1,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>","This is Tilly. She's just checking pup on you. Hopes you're doing ok. If not, she's available for pats, snugs, boops, the whole bit. 13/10 https://t.co/0Xxu71qeIV",NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421306343426/photo/1,13,10,Tilly,None,None,None,None


In [110]:
twitter_archive_df.tail(2)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
2354,666029285002620928,NaN,NaN,2015-11-15 23:05:30 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is a western brown Mitsubishi terrier. Upset about leaf. Actually 2 dogs here. 7/10 would walk the shit out of https://t.co/r7mOb2m0UI,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666029285002620928/photo/1,7,10,a,None,None,None,None
2355,666020888022790149,NaN,NaN,2015-11-15 22:32:08 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",Here we have a Japanese Irish Setter. Lost eye in Vietnam (?). Big fan of relaxing on stair. 8/10 would pet https://t.co/BLDqew2Ijj,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666020888022790149/photo/1,8,10,None,None,None,None,None


In [111]:
## download the image_prediction TSV file from the file using request

url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'
response = requests.get(url)

# Opening a tsv file and saving the response content
with open('image-predictions.tsv', mode='wb') as file:
    file.write(response.content)
    
# Read TSV file
image_prediction_df = pd.read_csv('image-predictions.tsv', sep='\t')
# Looking at the information in our data
image_prediction_df.head(3)

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True


In [112]:
### create an empty list, read the tweet_json file, use "for loop" to read it line by line.
### create a list of the required data from the json file

### create an empty list
more_data = []

## using os.path.expanduser to locate tweet-json text file
filepath_tweet = os.path.join(os.path.expanduser("~/Desktop/WRD"), "tweet_json.txt")

### open and create a dictionary using "for loop" to read file line by line from the "tweet_json.txt" to extract the tweet id, Retweet_count and Tweet_id  
with open(filepath_tweet, 'r', encoding = 'utf-8') as file:
    for line in file:
        each_tweet = json.loads(line)
        tweet_id = each_tweet['id']
        favorite_count = each_tweet['favorite_count']
        retweet_count = each_tweet['retweet_count']
        ### append tweets, 
        more_data.append({'tweet_id':tweet_id, 'retweet_count':retweet_count, 'favorite_count':favorite_count})

more_data## more_data is in dictionary  format

[{'tweet_id': 892420643555336193,
  'retweet_count': 8853,
  'favorite_count': 39467},
 {'tweet_id': 892177421306343426,
  'retweet_count': 6514,
  'favorite_count': 33819},
 {'tweet_id': 891815181378084864,
  'retweet_count': 4328,
  'favorite_count': 25461},
 {'tweet_id': 891689557279858688,
  'retweet_count': 8964,
  'favorite_count': 42908},
 {'tweet_id': 891327558926688256,
  'retweet_count': 9774,
  'favorite_count': 41048},
 {'tweet_id': 891087950875897856,
  'retweet_count': 3261,
  'favorite_count': 20562},
 {'tweet_id': 890971913173991426,
  'retweet_count': 2158,
  'favorite_count': 12041},
 {'tweet_id': 890729181411237888,
  'retweet_count': 16716,
  'favorite_count': 56848},
 {'tweet_id': 890609185150312448,
  'retweet_count': 4429,
  'favorite_count': 28226},
 {'tweet_id': 890240255349198849,
  'retweet_count': 7711,
  'favorite_count': 32467},
 {'tweet_id': 890006608113172480,
  'retweet_count': 7624,
  'favorite_count': 31166},
 {'tweet_id': 889880896479866881,
  'retwe

In [113]:
### convert tweet_api_data from dictionary to dataframe using panda
more_data_df = pd.DataFrame(more_data, columns=['tweet_id', 'retweet_count', 'favorite_count'])
more_data_df.head()

,tweet_id,retweet_count,favorite_count
0,892420643555336193,8853,39467
1,892177421306343426,6514,33819
2,891815181378084864,4328,25461
3,891689557279858688,8964,42908
4,891327558926688256,9774,41048


<a id='Assess'></a>
# **ASSESS**
Both Visual and Programmatic assessment would be done in this section

In this section I will be checking for the Quality and Tidiness of the these datasets


## **Visual assessment**

In [114]:
twitter_archive_df.sample(5) ## 

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
1497,692423280028966913,6.924173e+17,4.196984e+09,2016-01-27 19:05:49 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",PUPDATE: just noticed this dog has some extra legs. Very advanced. Revolutionary af. Upgraded to a 9/10,NaN,NaN,NaN,NaN,9,10,None,None,None,None,None
1265,709901256215666688,NaN,NaN,2016-03-16 00:37:03 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>","WeRateDogs stickers are here and they're 12/10! Use code ""puppers"" at checkout 🐶🐾\n\nShop now: https://t.co/k5xsufRKYm https://t.co/ShXk46V13r",NaN,NaN,NaN,"http://goo.gl/ArWZfi,https://twitter.com/dog_rates/status/709901256215666688/photo/1,https://twitter.com/dog_rates/status/709901256215666688/photo/1,https://twitter.com/dog_rates/status/709901256215666688/photo/1,https://twitter.com/dog_rates/status/709901256215666688/photo/1",12,10,None,None,None,None,None
1167,721503162398597120,NaN,NaN,2016-04-17 00:58:53 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Panda. He's happy af. 11/10 https://t.co/IOAk9i4UvE,NaN,NaN,NaN,"https://twitter.com/dog_rates/status/721503162398597120/photo/1,https://twitter.com/dog_rates/status/721503162398597120/photo/1,https://twitter.com/dog_rates/status/721503162398597120/photo/1,https://twitter.com/dog_rates/status/721503162398597120/photo/1",11,10,Panda,None,None,None,None
145,863553081350529029,NaN,NaN,2017-05-14 00:34:33 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Neptune. He's a backpup vocalist for the Dixie Chicks. 13/10 (vid by @AmiWinehouse) https://t.co/tordvmaaop,NaN,NaN,NaN,https://twitter.com/dog_rates/status/863553081350529029/video/1,13,10,Neptune,None,None,None,None
80,877316821321428993,NaN,NaN,2017-06-21 00:06:44 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>","Meet Dante. At first he wasn't a fan of his new raincoat, then he saw his reflection. H*ckin handsome. 13/10 for water resistant good boy https://t.co/SHRTIo5pxc",NaN,NaN,NaN,"https://twitter.com/dog_rates/status/877316821321428993/photo/1,https://twitter.com/dog_rates/status/877316821321428993/photo/1",13,10,Dante,None,None,None,None


In [115]:
image_prediction_df.sample(5)

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
27,666396247373291520,https://pbs.twimg.com/media/CT-D2ZHWIAA3gK1.jpg,1,Chihuahua,0.978108,True,toy_terrier,0.009397,True,papillon,0.004577,True
323,671874878652489728,https://pbs.twimg.com/media/CVL6op1WEAAUFE7.jpg,1,china_cabinet,0.996031,False,entertainment_center,0.001986,False,bookcase,0.001652,False
1994,874057562936811520,https://pbs.twimg.com/media/DCFGtdoXkAEsqIw.jpg,1,flat-coated_retriever,0.832177,True,black-and-tan_coonhound,0.040437,True,Newfoundland,0.028228,True
1064,715360349751484417,https://pbs.twimg.com/media/Ce14cOvWwAAcFJH.jpg,1,nail,0.855552,False,screw,0.073277,False,padlock,0.023970,False
1442,775364825476165632,https://pbs.twimg.com/media/CsKmMB2WAAAXcAy.jpg,3,beagle,0.571229,True,Chihuahua,0.175257,True,Pembroke,0.034306,True


In [116]:
more_data_df.sample(5)

,tweet_id,retweet_count,favorite_count
9,890240255349198849,7711,32467
1208,715680795826982913,1813,4719
943,752660715232722944,1794,4878
1801,676948236477857792,980,2354
2028,671789708968640512,3811,7527


## **Programmatic Assessment**

In [117]:
twitter_archive_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   tweet_id                    2356 non-null   int64  
 1   in_reply_to_status_id       78 non-null     float64
 2   in_reply_to_user_id         78 non-null     float64
 3   timestamp                   2356 non-null   object 
 4   source                      2356 non-null   object 
 5   text                        2356 non-null   object 
 6   retweeted_status_id         181 non-null    float64
 7   retweeted_status_user_id    181 non-null    float64
 8   retweeted_status_timestamp  181 non-null    object 
 9   expanded_urls               2297 non-null   object 
 10  rating_numerator            2356 non-null   int64  
 11  rating_denominator          2356 non-null   int64  
 12  name                        2356 non-null   object 
 13  doggo                       2356 

In [118]:
image_prediction_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   tweet_id  2075 non-null   int64  
 1   jpg_url   2075 non-null   object 
 2   img_num   2075 non-null   int64  
 3   p1        2075 non-null   object 
 4   p1_conf   2075 non-null   float64
 5   p1_dog    2075 non-null   bool   
 6   p2        2075 non-null   object 
 7   p2_conf   2075 non-null   float64
 8   p2_dog    2075 non-null   bool   
 9   p3        2075 non-null   object 
 10  p3_conf   2075 non-null   float64
 11  p3_dog    2075 non-null   bool   
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


In [119]:
more_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2354 entries, 0 to 2353
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype
---  ------          --------------  -----
 0   tweet_id        2354 non-null   int64
 1   retweet_count   2354 non-null   int64
 2   favorite_count  2354 non-null   int64
dtypes: int64(3)
memory usage: 55.3 KB


### count of rows and columns


In [120]:
twitter_archive_df.shape

(2356, 17)

In [121]:
image_prediction_df.shape

(2075, 12)

In [122]:
more_data_df.shape

(2354, 3)

### checking for duplicate rows and null cells.

In [123]:
twitter_archive_df.duplicated().all()

False

In [124]:
image_prediction_df.duplicated().all()

False

In [125]:
more_data_df.duplicated().all()

False

In [126]:
twitter_archive_df.isna().sum()

tweet_id                         0
in_reply_to_status_id         2278
in_reply_to_user_id           2278
timestamp                        0
source                           0
text                             0
retweeted_status_id           2175
retweeted_status_user_id      2175
retweeted_status_timestamp    2175
expanded_urls                   59
rating_numerator                 0
rating_denominator               0
name                             0
doggo                            0
floofer                          0
pupper                           0
puppo                            0
dtype: int64

In [127]:
image_prediction_df.isna().any()

tweet_id    False
jpg_url     False
img_num     False
p1          False
p1_conf     False
p1_dog      False
p2          False
p2_conf     False
p2_dog      False
p3          False
p3_conf     False
p3_dog      False
dtype: bool

In [128]:
more_data_df.isna().sum()

tweet_id          0
retweet_count     0
favorite_count    0
dtype: int64

In [129]:
total_null_cell = twitter_archive_df.isnull().sum().sort_values(ascending = False)
## --- total number of rows in dataframe
total_rows_count = twitter_archive_df.isnull().count().sort_values(ascending =False) 
## --- total number of null cells in each column

percentage = (total_null_cell/total_rows_count).sort_values(ascending= False)
empty_cells = pd.concat([total_null_cell, percentage], axis=1, keys=['Total_empty_cells', '%_empty_cells'])
empty_cells.style.format({'%_empty_cells': "{:.2%}"})

,Total_empty_cells,%_empty_cells
in_reply_to_user_id,2278,96.69%
in_reply_to_status_id,2278,96.69%
retweeted_status_timestamp,2175,92.32%
retweeted_status_id,2175,92.32%
retweeted_status_user_id,2175,92.32%
expanded_urls,59,2.50%
rating_denominator,0,0.00%
pupper,0,0.00%
floofer,0,0.00%
doggo,0,0.00%


### **More assessment**

###### for twitter_archive_df

In [130]:
twitter_archive_df.rating_numerator.value_counts().sort_values( ascending = False)

12      558
11      464
10      461
13      351
9       158
8       102
7        55
14       54
5        37
6        32
3        19
4        17
2         9
1         9
420       2
0         2
75        2
15        2
84        1
960       1
99        1
24        1
182       1
27        1
666       1
165       1
1776      1
204       1
17        1
50        1
45        1
60        1
44        1
143       1
121       1
20        1
26        1
144       1
80        1
88        1
Name: rating_numerator, dtype: int64

In [131]:
twitter_archive_df.rating_denominator.value_counts().sort_values( ascending = False)

10     2333
50        3
11        3
20        2
80        2
90        1
120       1
16        1
110       1
130       1
40        1
170       1
0         1
150       1
15        1
7         1
70        1
2         1
Name: rating_denominator, dtype: int64

In [132]:
### inspecting the name column.
twitter_archive_df.name

0        Phineas
1          Tilly
2         Archie
3          Darla
4       Franklin
          ...   
2351        None
2352           a
2353           a
2354           a
2355        None
Name: name, Length: 2356, dtype: object

In [133]:
### I visually inspected the name column and noticed that the bulk of the errors are in lowercase
### I will be inspecting it further programmatical

name_list = list(twitter_archive_df.name.unique()) ## create a list with all unique name in the name columns
name_error=[] ## this empty list will contain the names of lowercase which is predicted to be errorous
for n in name_list:
    if n[0].islower():
        name_error.append(n)
        print(n)        

such
a
quite
not
one
incredibly
mad
an
very
just
my
his
actually
getting
this
unacceptable
all
old
infuriating
the
by
officially
life
light
space


In [134]:
###  checking the content of source column  for any anomaly
twitter_archive_df.source.value_counts()

<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>     2221
<a href="http://vine.co" rel="nofollow">Vine - Make a Scene</a>                          91
<a href="http://twitter.com" rel="nofollow">Twitter Web Client</a>                       33
<a href="https://about.twitter.com/products/tweetdeck" rel="nofollow">TweetDeck</a>      11
Name: source, dtype: int64

In [135]:
### checking for the content of the text column

twitter_archive_df.text[1]

"This is Tilly. She's just checking pup on you. Hopes you're doing ok. If not, she's available for pats, snugs, boops, the whole bit. 13/10 https://t.co/0Xxu71qeIV"

In [136]:
twitter_archive_df.text[3]

'This is Darla. She commenced a snooze mid meal. 13/10 happens to the best of us https://t.co/tD36da7qLQ'

In [137]:
twitter_archive_df.text[60]

'This is Bella. She had her first beach experience this morning. Complete success. 12/10 would perform a sandy boop https://t.co/4VsFysDmiw'

In [138]:
twitter_archive_df.text.sample

<bound method NDFrame.sample of 0                                                            This is Phineas. He's a mystical boy. Only ever appears in the hole of a donut. 13/10 https://t.co/MgUWQ76dJU
1       This is Tilly. She's just checking pup on you. Hopes you're doing ok. If not, she's available for pats, snugs, boops, the whole bit. 13/10 https://t.co/0Xxu71qeIV
2                        This is Archie. He is a rare Norwegian Pouncing Corgo. Lives in the tall grass. You never know when one may strike. 12/10 https://t.co/wUnZnhtVJB
3                                                                  This is Darla. She commenced a snooze mid meal. 13/10 happens to the best of us https://t.co/tD36da7qLQ
4       This is Franklin. He would like you to stop calling him "cute." He is a very fierce shark and should be respected as such. 12/10 #BarkWeek https://t.co/AtUZn91f7f
                                                                                       ...                       

The text column contains both text and url

### For image_prediction_df

In [139]:
image_prediction_df.query("p1_dog== False & p2_dog== False & p3_dog == False ").count()

tweet_id    324
jpg_url     324
img_num     324
p1          324
p1_conf     324
p1_dog      324
p2          324
p2_conf     324
p2_dog      324
p3          324
p3_conf     324
p3_dog      324
dtype: int64

In [140]:
### checking jpg_url duplicate
image_prediction_df.jpg_url.value_counts()

#some url are repeated twice 

https://pbs.twimg.com/media/CZhn-QAWwAASQan.jpg                                            2
https://pbs.twimg.com/media/Cq9guJ5WgAADfpF.jpg                                            2
https://pbs.twimg.com/ext_tw_video_thumb/807106774843039744/pu/img/8XZg1xW35Xp2J6JW.jpg    2
https://pbs.twimg.com/media/CU1zsMSUAAAS0qW.jpg                                            2
https://pbs.twimg.com/media/CsrjryzWgAAZY00.jpg                                            2
                                                                                          ..
https://pbs.twimg.com/media/CXrmMSpUwAAdeRj.jpg                                            1
https://pbs.twimg.com/media/CXrawAhWkAAWSxC.jpg                                            1
https://pbs.twimg.com/media/CXrIntsUsAEkv0d.jpg                                            1
https://pbs.twimg.com/media/CXqcOHCUQAAugTB.jpg                                            1
https://pbs.twimg.com/media/DGKD1-bXoAAIAUK.jpg                       

In [141]:
## number of duplicated url in jpg_url columns
(image_prediction_df.jpg_url.duplicated()).value_counts()

False    2009
True       66
Name: jpg_url, dtype: int64

## Assessment observation  

>  ___Quality issue___ 

 
 **Twitter_archive_df**



 
1). timestamp has +0000 all through the column; It is unneccessary.

2). timestamp dtype is in string(object) instead of a datetime format.

3). text column contain not just text but some url.

4). missing names in the name column.

5). the values that are inputted in lowercase in the name column are not fit for a name such as "a".

6). some denominating rating are less than 10 and greater than 10.

7). some numerator rating are invalid.

8). source column contain text and url.

9). dogname contains none instead of NaN.

10). the retweet columns contains over.

11). some columns such as;(in_reply_to_user_id,in_reply_to_status_id,retweeted_status_timestamp,retweeted_status_id,retweeted_status_user_id) have over 90% of missing values.


--



**image_prediction_df**


12). inconsistent naming style for the breeds (p1, p2 and p3)

13). there are redundant columns in the image_prediction_df. It is not needed for the analysis.

14). All p1_dog == False rows is not needed for this analysis.

15). there are 66 duplicate entries in the jpg_url column

--
   

**more_data_df**

15). tweet_id is an integer instead of string.



> ___Tidiness issue___

1).  All tables can be merged to a single column.

2).  The dog category should be collapsed into a single column.





<a id='Clean'></a>
# **Clean**

In [142]:
## duplicate all dataframe
df_archive_clean = twitter_archive_df.copy()
df_image_clean = image_prediction_df.copy()
df_more_clean = more_data_df.copy()

In [143]:
df_archive_clean.head(3)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Phineas. He's a mystical boy. Only ever appears in the hole of a donut. 13/10 https://t.co/MgUWQ76dJU,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643555336193/photo/1,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>","This is Tilly. She's just checking pup on you. Hopes you're doing ok. If not, she's available for pats, snugs, boops, the whole bit. 13/10 https://t.co/0Xxu71qeIV",NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421306343426/photo/1,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Archie. He is a rare Norwegian Pouncing Corgo. Lives in the tall grass. You never know when one may strike. 12/10 https://t.co/wUnZnhtVJB,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181378084864/photo/1,12,10,Archie,None,None,None,None


In [144]:
df_image_clean.head()

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True


In [145]:
df_more_clean.head()

,tweet_id,retweet_count,favorite_count
0,892420643555336193,8853,39467
1,892177421306343426,6514,33819
2,891815181378084864,4328,25461
3,891689557279858688,8964,42908
4,891327558926688256,9774,41048


## **Quality issue**


### Issue 

The retweet columns contains over 96% missing value.

#### Define:

There are too many missing value in the retweet's columns which includes in_reply_to_user_id,in_reply_to_status_id,retweeted_status_timestamp,retweeted_status_id,retweeted_status_user_id). I will have to remove this as it will not help our analysis.

#### Code

In [146]:
# Drop all columns with replies, retweets and others
df_archive_clean

col = ['in_reply_to_status_id', 'in_reply_to_user_id', 
       'retweeted_status_id', 'retweeted_status_user_id', 
       'retweeted_status_timestamp', 'expanded_urls']
df_archive_clean.drop(col, axis=1, inplace=True)

#### Test

In [147]:
df_archive_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   tweet_id            2356 non-null   int64 
 1   timestamp           2356 non-null   object
 2   source              2356 non-null   object
 3   text                2356 non-null   object
 4   rating_numerator    2356 non-null   int64 
 5   rating_denominator  2356 non-null   int64 
 6   name                2356 non-null   object
 7   doggo               2356 non-null   object
 8   floofer             2356 non-null   object
 9   pupper              2356 non-null   object
 10  puppo               2356 non-null   object
dtypes: int64(3), object(8)
memory usage: 202.6+ KB


### Issue
 Timestamp has +0000 all through the column; It is unneccessary.
 
 #### Define
  I will remove the +0000 in the timestamp column.


#### Code

In [148]:
df_archive_clean.timestamp=df_archive_clean.timestamp.apply(lambda x: x[:-5])

### Test

In [149]:
df_archive_clean.timestamp.sample(3)

1492    2016-01-28 21:54:41 
1456    2016-02-04 18:35:39 
285     2017-03-07 00:57:32 
Name: timestamp, dtype: object

### Issue

Timestamp dtype is in string(object) instead of a datetime format.


#### Define:

The Timestamp datatype in Twitter_Archive_df is a string. Time column are to be standardized as "datetime" format.

#### Code

In [150]:
df_archive_clean.timestamp = pd.to_datetime(df_archive_clean.timestamp)

#### Test

In [151]:
df_archive_clean.timestamp.dtype

dtype('<M8[ns]')

### Issue
Date and time are in the same column(timestamp)
#### Define:
The date and time should be in seperate columns; I will be creating new columns for both Time and Date while I drop the timestamp column.


#### Code

In [152]:
## create date column
df_archive_clean['date'] = df_archive_clean.timestamp.dt.date

## create time column
df_archive_clean['time'] = df_archive_clean.timestamp.dt.time 

## convert from object dtype to datetime
df_archive_clean.date = pd.to_datetime(df_archive_clean.date)

## drop the timestamp column
df_archive_clean.drop(columns = ['timestamp'], inplace = True)

#### Test

In [153]:
df_archive_clean.info()

df_archive_clean.head(1)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   tweet_id            2356 non-null   int64         
 1   source              2356 non-null   object        
 2   text                2356 non-null   object        
 3   rating_numerator    2356 non-null   int64         
 4   rating_denominator  2356 non-null   int64         
 5   name                2356 non-null   object        
 6   doggo               2356 non-null   object        
 7   floofer             2356 non-null   object        
 8   pupper              2356 non-null   object        
 9   puppo               2356 non-null   object        
 10  date                2356 non-null   datetime64[ns]
 11  time                2356 non-null   object        
dtypes: datetime64[ns](1), int64(3), object(8)
memory usage: 221.0+ KB


,tweet_id,source,text,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo,date,time
0,892420643555336193,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Phineas. He's a mystical boy. Only ever appears in the hole of a donut. 13/10 https://t.co/MgUWQ76dJU,13,10,Phineas,None,None,None,None,2017-08-01,16:23:56


### Issue

Text column contain not just text but some url.

#### Define:
The text column in the Twitter_archive_df contains both the tweet text and some Url. I will be extracting the Text only.


#### Code

In [154]:
def text_only(a):
    a = a.split('https:')[0]
    return a ## create a function to split text from URL .

df_archive_clean['text'] = df_archive_clean['text'].apply(text_only) ## apply the function text_only

#### Test

In [155]:
for content in df_archive_clean.text.sample(3):
    print(content)

Just received another perfect photo of dogs and the sunset. 12/10 
Say hello to Clarence. He's a western Alkaline Pita. Very proud of himself for dismembering his stuffed dog pal 8/10 
This is Ralphé. He patrols the lake. Looking for babes. 11/10 


### Issue

Source column contain text and url.

#### Define:

The "source" column in the twitter_archive_df should contain only the "source/device" from which the tweet was made by the user. The values in this column is stored in a url format. I will be extracting the source/device from these urls  

#### Code

In [156]:
df_archive_clean.source=df_archive_clean.source.str.extract(r'\>(.*?)\<')

#### Test

In [157]:
df_archive_clean.source.value_counts()

Twitter for iPhone     2221
Vine - Make a Scene      91
Twitter Web Client       33
TweetDeck                11
Name: source, dtype: int64

#### Issue
Source column datatype is in Object

#### Define: 
The "Source" column is a fixed Categorical variable without middle ground intervals. It is either this or that. This column would be changed to a category datatype for better optimization and analysis.


#### Code

In [158]:
df_archive_clean.source = df_archive_clean['source'].astype('category')

#### Test

In [159]:
df_archive_clean["source"].dtype

CategoricalDtype(categories=['TweetDeck', 'Twitter Web Client', 'Twitter for iPhone',
                  'Vine - Make a Scene'],
, ordered=False)

In [160]:
df_archive_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   tweet_id            2356 non-null   int64         
 1   source              2356 non-null   category      
 2   text                2356 non-null   object        
 3   rating_numerator    2356 non-null   int64         
 4   rating_denominator  2356 non-null   int64         
 5   name                2356 non-null   object        
 6   doggo               2356 non-null   object        
 7   floofer             2356 non-null   object        
 8   pupper              2356 non-null   object        
 9   puppo               2356 non-null   object        
 10  date                2356 non-null   datetime64[ns]
 11  time                2356 non-null   object        
dtypes: category(1), datetime64[ns](1), int64(3), object(7)
memory usage: 205.1+ KB


### Issue
Tweet_id is an integer instead of string.

#### Define
The tweet_id is a unique identifier(primary key). It should be best preserved in a string format rather int to avoid problem.  


#### Code

In [161]:
df_archive_clean.tweet_id = df_archive_clean.tweet_id.astype(str)
df_image_clean.tweet_id = df_image_clean.tweet_id.astype(str)
df_more_clean.tweet_id = df_more_clean.tweet_id.astype(str)


#### Test

In [162]:
df_archive_clean.dtypes

tweet_id                      object
source                      category
text                          object
rating_numerator               int64
rating_denominator             int64
name                          object
doggo                         object
floofer                       object
pupper                        object
puppo                         object
date                  datetime64[ns]
time                          object
dtype: object

In [163]:
df_image_clean.dtypes

tweet_id     object
jpg_url      object
img_num       int64
p1           object
p1_conf     float64
p1_dog         bool
p2           object
p2_conf     float64
p2_dog         bool
p3           object
p3_conf     float64
p3_dog         bool
dtype: object

In [164]:
df_more_clean.dtypes

tweet_id          object
retweet_count      int64
favorite_count     int64
dtype: object

In [165]:
df_archive_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   tweet_id            2356 non-null   object        
 1   source              2356 non-null   category      
 2   text                2356 non-null   object        
 3   rating_numerator    2356 non-null   int64         
 4   rating_denominator  2356 non-null   int64         
 5   name                2356 non-null   object        
 6   doggo               2356 non-null   object        
 7   floofer             2356 non-null   object        
 8   pupper              2356 non-null   object        
 9   puppo               2356 non-null   object        
 10  date                2356 non-null   datetime64[ns]
 11  time                2356 non-null   object        
dtypes: category(1), datetime64[ns](1), int64(2), object(8)
memory usage: 205.1+ KB


### Issue
Some rows in the name column are wrongly captured.
#### Define 
The name column was extracted from the "text" column thus, some are wrongly captured ( e.g This is....). Ideally, this rows should contain a NaN value. I will be filling this rows with a NaN value

#### Code

In [166]:
## Re extract name from text column properly by taking the Word after "This is" ( most likely the  dog name) 

index_text_for_name = re.compile(r'(?:name(?:d)?)\s{1}(?:is\s)?([A-Za-z]+)') ## extract name from text column using regex
for index, row in df_archive_clean.iterrows():  
    if (row['name'].islower()):
        try:
            new_name = re.findall(index_text_for_name , row['text'])[0]
            df_archive_clean.loc[index,'name'] = df_archive_clean.loc[index,'name'].replace(row['name'], new_name)
        except IndexError:
            df_archive_clean.loc[index,'name'] = np.nan
## Replace none value with Nan
df_archive_clean['name'] = df_archive_clean['name'].replace('None', np.nan)

#### Test  

In [167]:
## number of rows that has None in the name column
df_archive_clean[(df_archive_clean.name) == 'None'].count()

tweet_id              0
source                0
text                  0
rating_numerator      0
rating_denominator    0
name                  0
doggo                 0
floofer               0
pupper                0
puppo                 0
date                  0
time                  0
dtype: int64

In [168]:
len(df_archive_clean[df_archive_clean.name.isin(name_error)])
0

0

In [169]:
for n in df_archive_clean.name:
    if n == "None":
        print(n)

In [170]:
df_archive_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   tweet_id            2356 non-null   object        
 1   source              2356 non-null   category      
 2   text                2356 non-null   object        
 3   rating_numerator    2356 non-null   int64         
 4   rating_denominator  2356 non-null   int64         
 5   name                1524 non-null   object        
 6   doggo               2356 non-null   object        
 7   floofer             2356 non-null   object        
 8   pupper              2356 non-null   object        
 9   puppo               2356 non-null   object        
 10  date                2356 non-null   datetime64[ns]
 11  time                2356 non-null   object        
dtypes: category(1), datetime64[ns](1), int64(2), object(8)
memory usage: 205.1+ KB


### Issue
Some numerator rating is invalid

#### Define
The numerator ratings are extracted from the text columns. The method of indexing has caused it to pick up some wrong number for rating, the numbers are typically >10. I will be manually correcting this errors.

####  Code

In [171]:
## Observe columns visually
pd.set_option("display.max_colwidth", None) ### display the full text on output
df_archive_clean.loc[(df_archive_clean.rating_numerator > 15), ['tweet_id', 'rating_numerator', 'rating_denominator', 'text']]

,tweet_id,rating_numerator,rating_denominator,text
55,881633300179243008,17,10,@roushfenway These are good dogs but 17/10 is an emotional impulse rating. More like 13/10s
188,855862651834028034,420,10,@dhmontgomery We also gave snoop dogg a 420/10 but I think that predated your research
189,855860136149123072,666,10,"@s8n You tried very hard to portray this good boy as not so good, but you have ultimately failed. His goodness shines through. 666/10"
290,838150277551247360,182,10,@markhoppus 182/10
313,835246439529840640,960,0,"@jonnysun @Lin_Manuel ok jomny I know you're excited but 960/00 isn't a valid rating, 13/10 is tho"
340,832215909146226688,75,10,"RT @dog_rates: This is Logan, the Chow who lived. He solemnly swears he's up to lots of good. H*ckin magical af 9.75/10"
433,820690176645140481,84,70,The floofs have been released I repeat the floofs have been released. 84/70
516,810984652412424192,24,7,Meet Sam. She smiles 24/7 &amp; secretly aspires to be a reindeer. \nKeep Sam smiling by clicking and sharing this link:\n
695,786709082849828864,75,10,"This is Logan, the Chow who lived. He solemnly swears he's up to lots of good. H*ckin magical af 9.75/10"
763,778027034220126208,27,10,This is Sophie. She's a Jubilant Bush Pupper. Super h*ckin rare. Appears at random just to smile at the locals. 11.27/10 would smile back


In [172]:
## all rows with no actual rating rating in text column is ascribe NaN value.
df_archive_clean.loc[[188,189,290,433,516,902,979,1120,1202,1228,1254,1274,1351,1433,1634,1635,1663,1779,1843,2074], ['rating_numerator','rating_denominator']] = np.NaN

#### Test

In [173]:
## Test
df_nan = df_archive_clean.loc[[188,189,290,433,516,902,979,1120,1202,1228,1254,1274,1351,1433,1634,1635,1663,1779,1843,2074], ['rating_numerator','rating_denominator']]
df_nan 

,rating_numerator,rating_denominator
188,NaN,NaN
189,NaN,NaN
290,NaN,NaN
433,NaN,NaN
516,NaN,NaN
902,NaN,NaN
979,NaN,NaN
1120,NaN,NaN
1202,NaN,NaN
1228,NaN,NaN


#### Code 
- Correcting the rows in the  rating_numerator column with actual rating in its text column that is misplaced.

- dropping the Null rows from the table.


In [174]:
## rows containing actual rating in the text column.
df_archive_clean.loc[(df_archive_clean.rating_numerator > 15), ['tweet_id', 'rating_numerator', 'rating_denominator', 'text']]

,tweet_id,rating_numerator,rating_denominator,text
55,881633300179243008,17.0,10.0,@roushfenway These are good dogs but 17/10 is an emotional impulse rating. More like 13/10s
313,835246439529840640,960.0,0.0,"@jonnysun @Lin_Manuel ok jomny I know you're excited but 960/00 isn't a valid rating, 13/10 is tho"
340,832215909146226688,75.0,10.0,"RT @dog_rates: This is Logan, the Chow who lived. He solemnly swears he's up to lots of good. H*ckin magical af 9.75/10"
695,786709082849828864,75.0,10.0,"This is Logan, the Chow who lived. He solemnly swears he's up to lots of good. H*ckin magical af 9.75/10"
763,778027034220126208,27.0,10.0,This is Sophie. She's a Jubilant Bush Pupper. Super h*ckin rare. Appears at random just to smile at the locals. 11.27/10 would smile back
1712,680494726643068929,26.0,10.0,Here we have uncovered an entire battalion of holiday puppers. Average of 11.26/10


In [175]:
## manually extracting the correct numerator rating from text column.
df_archive_clean.loc[55,'rating_numerator'] = 13
df_archive_clean.loc[313,'rating_numerator'] = 13
df_archive_clean.loc[340,'rating_numerator'] = 10
df_archive_clean.loc[695,'rating_numerator'] = 10
df_archive_clean.loc[763,'rating_numerator'] = 11
df_archive_clean.loc[1712,'rating_numerator'] = 11

In [176]:
df_archive_clean.loc[(df_archive_clean.rating_numerator > 15), ['tweet_id', 'rating_numerator', 'rating_denominator', 'text']]

df_archive_clean.dropna(subset=['rating_numerator'], how ='all', inplace = True)

#### Test

In [177]:
## check null value 
df_archive_clean.rating_numerator.isna().any()

False

In [178]:
df_archive_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2336 entries, 0 to 2355
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   tweet_id            2336 non-null   object        
 1   source              2336 non-null   category      
 2   text                2336 non-null   object        
 3   rating_numerator    2336 non-null   float64       
 4   rating_denominator  2336 non-null   float64       
 5   name                1521 non-null   object        
 6   doggo               2336 non-null   object        
 7   floofer             2336 non-null   object        
 8   pupper              2336 non-null   object        
 9   puppo               2336 non-null   object        
 10  date                2336 non-null   datetime64[ns]
 11  time                2336 non-null   object        
dtypes: category(1), datetime64[ns](1), float64(2), object(8)
memory usage: 221.5+ KB


In [179]:
## Test if index still exist in column.
try:
    df_nan = df_archive_clean.loc[[188,189,290,433,516,902,979,1120,1202,1228,1254,1274,1351,1433,1634,1635,1663,1779,1843,2074], ['rating_numerator','rating_denominator']]
except KeyError:
    print("index doesn't exist in column")

index doesn't exist in column


In [180]:
df_archive_clean.sample(5)

,tweet_id,source,text,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo,date,time
1362,703041949650034688,Twitter for iPhone,This is an East African Chalupa Seal. We only rate dogs. Please only send in dogs. Thank you... 10/10,10.0,10.0,NaN,None,None,None,None,2016-02-26,02:20:37
166,859924526012018688,Twitter for iPhone,Meet Milky. She has no idea what happened. Just as pupset as you. Perhaps a sheep exploded. Even offered to help clean. 12/10 very good girl,12.0,10.0,Milky,None,None,None,None,2017-05-04,00:15:58
1848,675798442703122432,Twitter for iPhone,This is Bernie. He just touched a boob for the first time. 10/10,10.0,10.0,Bernie,None,None,None,None,2015-12-12,22:04:39
552,804413760345620481,Twitter for iPhone,RT @dog_rates: This is Rusty. He's going D1 for sure. Insane vertical. 13/10 would draft,13.0,10.0,Rusty,None,None,None,None,2016-12-01,19:56:00
275,840696689258311684,Twitter for iPhone,I didn't even have to intervene. Took him 4 minutes to realize his error. 10/10 for Kevin,10.0,10.0,NaN,None,None,None,None,2017-03-11,22:51:24


## **Image Prediction Cleaning**


12). inconsistent naming style for the breeds (p1, p2 and p3)

13). there are redundant columns in the image_prediction_df. It is not needed for the analysis.

14). All p1_dog == False rows is not needed for this analysis.

15). there are 66 duplicate entries in the jpg_url column


In [181]:
df_image_clean.sample(5)

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
49,666835007768551424,https://pbs.twimg.com/media/CUES51dXIAEahyG.jpg,1,Airedale,0.448459,True,toy_poodle,0.124030,True,teddy,0.110183,False
1344,759099523532779520,https://pbs.twimg.com/media/Cojc_Q0WcAAqi_K.jpg,1,Shetland_sheepdog,0.129034,True,kelpie,0.117508,True,Siberian_husky,0.106708,True
1268,749417653287129088,https://pbs.twimg.com/media/CmZ3YH9WEAAowi3.jpg,2,papillon,0.772894,True,Shetland_sheepdog,0.042408,True,collie,0.042313,True
1986,872486979161796608,https://pbs.twimg.com/media/DBuyRlTUwAAYhG9.jpg,1,Pembroke,0.931861,True,Cardigan,0.037721,True,Chihuahua,0.011967,True
1819,834209720923721728,https://pbs.twimg.com/media/C5O1UAaWIAAMBMd.jpg,1,golden_retriever,0.754799,True,Pekinese,0.197861,True,Labrador_retriever,0.008654,True


### Issue
Inconsistent naming style for the breeds (p1, p2 and p3)

#### Define
The image prediction columns for the dog breed prediction is inconsistent; I will be unifying it by replacing the spaces with underscore and also changing all uppercase strings to lowercase.


#### Code

In [182]:
df_image_clean['p1'].unique()

array(['Welsh_springer_spaniel', 'redbone', 'German_shepherd',
       'Rhodesian_ridgeback', 'miniature_pinscher',
       'Bernese_mountain_dog', 'box_turtle', 'chow', 'shopping_cart',
       'miniature_poodle', 'golden_retriever', 'Gordon_setter',
       'Walker_hound', 'pug', 'bloodhound', 'Lhasa', 'English_setter',
       'hen', 'desktop_computer', 'Italian_greyhound', 'Maltese_dog',
       'three-toed_sloth', 'ox', 'malamute', 'guinea_pig',
       'soft-coated_wheaten_terrier', 'Chihuahua',
       'black-and-tan_coonhound', 'coho', 'toy_terrier',
       'Blenheim_spaniel', 'Pembroke', 'llama',
       'Chesapeake_Bay_retriever', 'curly-coated_retriever', 'dalmatian',
       'Ibizan_hound', 'Border_collie', 'Labrador_retriever', 'seat_belt',
       'snail', 'miniature_schnauzer', 'Airedale', 'triceratops', 'swab',
       'hay', 'hyena', 'jigsaw_puzzle', 'West_Highland_white_terrier',
       'toy_poodle', 'giant_schnauzer', 'vizsla', 'vacuum', 'Rottweiler',
       'Siberian_husky', 't

In [183]:
df_image_clean['p2'].unique()

array(['collie', 'miniature_pinscher', 'malinois', 'redbone',
       'Rottweiler', 'English_springer', 'mud_turtle', 'Tibetan_mastiff',
       'shopping_basket', 'komondor', 'Yorkshire_terrier',
       'English_foxhound', 'bull_mastiff', 'German_shepherd', 'Shih-Tzu',
       'Newfoundland', 'cock', 'desk', 'toy_terrier', 'toy_poodle',
       'otter', 'Chesapeake_Bay_retriever', 'Siberian_husky', 'skunk',
       'Afghan_hound', 'bloodhound', 'barracouta', 'papillon',
       'cocker_spaniel', 'chow', 'Irish_terrier', 'chain_saw', 'beagle',
       'giant_schnauzer', 'Labrador_retriever', 'Pembroke', 'Chihuahua',
       'Weimaraner', 'slug', 'Brittany_spaniel', 'standard_schnauzer',
       'teddy', 'armadillo', 'African_hunting_dog', 'vizsla', 'doormat',
       'pug', 'Italian_greyhound', 'Samoyed', 'Pomeranian',
       'miniature_poodle', 'Lakeland_terrier', 'Irish_setter', 'swab',
       'malamute', 'bath_towel', 'Border_collie', 'Leonberg', 'drake',
       'French_bulldog', 'ice_bear', 

In [184]:
df_image_clean['p3'].unique()

array(['Shetland_sheepdog', 'Rhodesian_ridgeback', 'bloodhound',
       'miniature_pinscher', 'Doberman', 'Greater_Swiss_Mountain_dog',
       'terrapin', 'fur_coat', 'golden_retriever',
       'soft-coated_wheaten_terrier', 'Labrador_retriever', 'Pekinese',
       'Ibizan_hound', 'French_bulldog', 'malinois', 'Dandie_Dinmont',
       'borzoi', 'partridge', 'bookcase', 'basenji', 'miniature_poodle',
       'great_grey_owl', 'groenendael', 'Eskimo_dog', 'hamster', 'briard',
       'papillon', 'flat-coated_retriever', 'gar', 'Chihuahua',
       'Shih-Tzu', 'Pomeranian', 'dingo', 'power_drill', 'Saluki',
       'Great_Pyrenees', 'West_Highland_white_terrier', 'collie',
       'toy_poodle', 'vizsla', 'acorn', 'giant_schnauzer', 'teddy',
       'common_iguana', 'wig', 'water_buffalo', 'coyote', 'seat_belt',
       'kelpie', 'space_heater', 'Brabancon_griffon', 'standard_poodle',
       'beagle', 'Irish_water_spaniel', 'bluetick', 'Weimaraner',
       'Chesapeake_Bay_retriever', 'toilet_tiss

In [185]:
## using lambda to replace underscore.

df_image_clean.p1 = list(map(lambda x: x.replace(' ', '_'),
                                 df_image_clean.p1))
df_image_clean.p2 = list(map(lambda x: x.replace(' ', '_'),
                                 df_image_clean.p2))
df_image_clean.p3 = list(map(lambda x: x.replace(' ', '_'),
                                 df_image_clean.p3))


### Replace all uppercase with lowercase character
df_image_clean.p1 = df_image_clean.p1.str.lower()
df_image_clean.p2 = df_image_clean.p2.str.lower()
df_image_clean.p3 = df_image_clean.p3.str.lower()

#### Test

In [186]:
df_image_clean['p1'].unique()

array(['welsh_springer_spaniel', 'redbone', 'german_shepherd',
       'rhodesian_ridgeback', 'miniature_pinscher',
       'bernese_mountain_dog', 'box_turtle', 'chow', 'shopping_cart',
       'miniature_poodle', 'golden_retriever', 'gordon_setter',
       'walker_hound', 'pug', 'bloodhound', 'lhasa', 'english_setter',
       'hen', 'desktop_computer', 'italian_greyhound', 'maltese_dog',
       'three-toed_sloth', 'ox', 'malamute', 'guinea_pig',
       'soft-coated_wheaten_terrier', 'chihuahua',
       'black-and-tan_coonhound', 'coho', 'toy_terrier',
       'blenheim_spaniel', 'pembroke', 'llama',
       'chesapeake_bay_retriever', 'curly-coated_retriever', 'dalmatian',
       'ibizan_hound', 'border_collie', 'labrador_retriever', 'seat_belt',
       'snail', 'miniature_schnauzer', 'airedale', 'triceratops', 'swab',
       'hay', 'hyena', 'jigsaw_puzzle', 'west_highland_white_terrier',
       'toy_poodle', 'giant_schnauzer', 'vizsla', 'vacuum', 'rottweiler',
       'siberian_husky', 't

### Issue
Non-dog images are present in the image prediction column.
#### Define
I will query all the false image prediction in the column and drop them.

#### Code


In [187]:
df_image_clean.sample(10)

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
1173,737310737551491075,https://pbs.twimg.com/ext_tw_video_thumb/737310236135043073/pu/img/_lG4DXmH-_XEq7Rc.jpg,1,cliff,0.439077,False,lakeside,0.062899,False,valley,0.039758,False
922,702217446468493312,https://pbs.twimg.com/media/Cb7HCMkWEAAV9zY.jpg,1,golden_retriever,0.242419,True,chow,0.226800,True,cocker_spaniel,0.194086,True
770,689289219123089408,https://pbs.twimg.com/ext_tw_video_thumb/689289176076959744/pu/img/hEFkFtmMu_hkTlxK.jpg,1,snowmobile,0.254642,False,assault_rifle,0.129558,False,rifle,0.110875,False
667,682962037429899265,https://pbs.twimg.com/media/CXpeVzQW8AApKYb.jpg,1,dingo,0.278600,False,chihuahua,0.155207,True,loupe,0.153598,False
1886,847962785489326080,https://pbs.twimg.com/media/C8SRpHNUIAARB3j.jpg,1,sea_lion,0.882654,False,mink,0.066880,False,otter,0.025679,False
900,699801817392291840,https://pbs.twimg.com/media/CbYyCMcWIAAHHjF.jpg,2,golden_retriever,0.808978,True,irish_setter,0.042428,True,labrador_retriever,0.023536,True
1946,862457590147678208,https://pbs.twimg.com/media/C_gQmaTUMAAPYSS.jpg,1,home_theater,0.496348,False,studio_couch,0.167256,False,barber_chair,0.052625,False
709,685268753634967552,https://pbs.twimg.com/media/CYKQS0xUQAEOptC.jpg,1,pug,0.999044,True,norwegian_elkhound,0.000547,True,bull_mastiff,0.000235,True
1570,794926597468000259,https://pbs.twimg.com/media/CwglhZVXgAAc3_w.jpg,1,teddy,0.569566,False,bath_towel,0.173745,False,toy_poodle,0.037662,True
1008,709207347839836162,https://pbs.twimg.com/media/CdecUSzUIAAHCvg.jpg,1,chihuahua,0.948323,True,italian_greyhound,0.017730,True,quilt,0.016688,False


In [188]:
### not_dog for rows with all prediction is dog(False)
not_dog = df_image_clean.query("p1_dog == False & p2_dog == False & p3_dog == False").index
df_image_clean.drop(not_dog,inplace = True)

#### Test

In [189]:
len(df_image_clean.query('p1_dog == False & p2_dog == False & p3_dog == False'))

0

### Issue
Different tweet_ids have same jpg_url

#### Define 
Some Tweet_ids contains same jpg_url which could be due to retweets. Duplicated images will not be needed for this analysis, therefore, it will be dropped from the table.

In [190]:
df_image_clean['jpg_url'].duplicated().sum()

60

In [191]:
### drop duplictated URL
df_image_clean.drop_duplicates('jpg_url', inplace = True)

#### Test

In [192]:
df_image_clean['jpg_url'].duplicated().sum()

0

### Issue
The prediction models picked up some more obvious object in the picture.
#### Define
The first prediction (p1) has the highest prediction confidence (p1_conf) with records but it could pick up more obvious object which might not be dog, which render the "p1_dog" FALSE. I will create a column for the likely dog breed using one of the Predictions that are true.




#### Code

In [193]:
### Create a function for the most likely breed of dog
def breed_pred(x):
    if x[0]==True:
        return x[1]
    elif x[2]==True:
        return x[3]
    else:
        return x[4]
    
df_image_clean['breed'] = df_image_clean[['p1_dog', 'p1', 'p2_dog', 'p2', 'p3', 'p3_dog']].apply(breed_pred, axis=1)   

#### Test

In [194]:
df_image_clean.head(10)

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog,breed
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,welsh_springer_spaniel,0.465074,True,collie,0.156665,True,shetland_sheepdog,0.061428,True,welsh_springer_spaniel
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,rhodesian_ridgeback,0.072010,True,redbone
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,german_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True,german_shepherd
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True,rhodesian_ridgeback
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,rottweiler,0.243682,True,doberman,0.154629,True,miniature_pinscher
5,666050758794694657,https://pbs.twimg.com/media/CT5Jof1WUAEuVxN.jpg,1,bernese_mountain_dog,0.651137,True,english_springer,0.263788,True,greater_swiss_mountain_dog,0.016199,True,bernese_mountain_dog
7,666055525042405380,https://pbs.twimg.com/media/CT5N9tpXIAAifs1.jpg,1,chow,0.692517,True,tibetan_mastiff,0.058279,True,fur_coat,0.054449,False,chow
8,666057090499244032,https://pbs.twimg.com/media/CT5PY90WoAAQGLo.jpg,1,shopping_cart,0.962465,False,shopping_basket,0.014594,False,golden_retriever,0.007959,True,golden_retriever
9,666058600524156928,https://pbs.twimg.com/media/CT5Qw94XAAA_2dP.jpg,1,miniature_poodle,0.201493,True,komondor,0.192305,True,soft-coated_wheaten_terrier,0.082086,True,miniature_poodle
10,666063827256086533,https://pbs.twimg.com/media/CT5Vg_wXIAAXfnj.jpg,1,golden_retriever,0.775930,True,tibetan_mastiff,0.093718,True,labrador_retriever,0.072427,True,golden_retriever


###  Issue
Unwanted image prediction confidence.


### Define
There are three image prediction columns in the table, I will create a column " p_conf" that will take is value with dependence with the corresponding conf level from with the breed column takes it value.

#### Code

In [195]:
def get_dependent_column(row):
    if row['p1'] == row['breed']:
        return row['p1_conf']
    elif row['p2'] == row['breed']:
        return row['p2_conf']
    elif row['p3'] == row['breed']:
        return row['p3_conf']
    else:
        return None
df_image_clean['p_conf'] = df_image_clean.apply(lambda row: get_dependent_column(row),axis = 1)

#### Test

In [196]:
df_image_clean.sample(5)

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog,breed,p_conf
125,668248472370458624,https://pbs.twimg.com/media/CUYYcMfXAAAixe7.jpg,1,chihuahua,0.734547,True,miniature_pinscher,0.068294,True,toy_terrier,0.046367,True,chihuahua,0.734547
1865,843235543001513987,https://pbs.twimg.com/media/C7PGQJAWwAAibui.jpg,1,pembroke,0.958452,True,cardigan,0.023770,True,chihuahua,0.005269,True,pembroke,0.958452
1468,778990705243029504,https://pbs.twimg.com/media/Cs-H5uhWcAAiNY9.jpg,2,cocker_spaniel,0.715351,True,labrador_retriever,0.207056,True,chihuahua,0.028519,True,cocker_spaniel,0.715351
1763,826240494070030336,https://pbs.twimg.com/media/C3dlVMbXAAUd-Gh.jpg,1,french_bulldog,0.903048,True,pug,0.096242,True,boston_bull,0.000234,True,french_bulldog,0.903048
1245,747512671126323200,https://pbs.twimg.com/media/Cl-yykwWkAAqUCE.jpg,1,cardigan,0.111493,True,malinois,0.095089,True,german_shepherd,0.080146,True,cardigan,0.111493


### Issue

There are redundant columns in the image_prediction table.

### Define

Drop all redundant column from the dataframe. 

#### Code

In [197]:
## drop redundant columns in image_clean

df_image_clean.drop(columns =['img_num', 'p1', 'p1_conf', 'p1_dog', 'p2', 'p2_conf', 'p2_dog', 'p3', 'p3_conf', 'p3_dog'], inplace = True) 

#### Test

In [198]:
df_image_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1691 entries, 0 to 2073
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   tweet_id  1691 non-null   object 
 1   jpg_url   1691 non-null   object 
 2   breed     1691 non-null   object 
 3   p_conf    1691 non-null   float64
dtypes: float64(1), object(3)
memory usage: 66.1+ KB


## **df_more_clean**

In [199]:
df_more_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2354 entries, 0 to 2353
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   tweet_id        2354 non-null   object
 1   retweet_count   2354 non-null   int64 
 2   favorite_count  2354 non-null   int64 
dtypes: int64(2), object(1)
memory usage: 55.3+ KB


In [200]:
df_more_clean.duplicated().any()

False

The df_more_clean dataset looks good.

In [201]:
df_archive_clean.info()
df_image_clean.info()
df_more_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2336 entries, 0 to 2355
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   tweet_id            2336 non-null   object        
 1   source              2336 non-null   category      
 2   text                2336 non-null   object        
 3   rating_numerator    2336 non-null   float64       
 4   rating_denominator  2336 non-null   float64       
 5   name                1521 non-null   object        
 6   doggo               2336 non-null   object        
 7   floofer             2336 non-null   object        
 8   pupper              2336 non-null   object        
 9   puppo               2336 non-null   object        
 10  date                2336 non-null   datetime64[ns]
 11  time                2336 non-null   object        
dtypes: category(1), datetime64[ns](1), float64(2), object(8)
memory usage: 286.0+ KB
<class 'pandas.core.fra

# **Tidiness Issue**

## Issue
The dog stage columns(doggo, floofer, pupper, puppo) should be collapsed into a single column stage


## Define 
I will interate through the  four columns and join the stage name into one single column stage. For rows that are empty, it will be converted to None value and the none value will be removed. 

### Code

In [202]:
## create column 'stage' by merging the four dog stages columns, replacing the empty rows wiht nan values

df_archive_clean['stage'] = df_archive_clean[['doggo', 'floofer', 'pupper', 'puppo']].apply(lambda x: ''.join(x), axis=1)
df_archive_clean['stage'].replace('', np.nan, inplace=True)
value_counts = df_archive_clean['stage'].value_counts()

value_counts



NoneNoneNoneNone        1956
NoneNonepupperNone       245
doggoNoneNoneNone         83
NoneNoneNonepuppo         29
doggoNonepupperNone       12
NoneflooferNoneNone        9
doggoNoneNonepuppo         1
doggoflooferNoneNone       1
Name: stage, dtype: int64

In [203]:
# Drop rows with 'None' in the 'dog_stage' column
df_archive_clean = df_archive_clean[df_archive_clean['stage'] != 'None']

df_archive_clean['stage'] = df_archive_clean['stage'].map(lambda x: x.replace("None",""))

### Test

In [204]:
df_archive_clean['stage'].value_counts()

                1956
pupper           245
doggo             83
puppo             29
doggopupper       12
floofer            9
doggopuppo         1
doggofloofer       1
Name: stage, dtype: int64

In [205]:
df_archive_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2336 entries, 0 to 2355
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   tweet_id            2336 non-null   object        
 1   source              2336 non-null   category      
 2   text                2336 non-null   object        
 3   rating_numerator    2336 non-null   float64       
 4   rating_denominator  2336 non-null   float64       
 5   name                1521 non-null   object        
 6   doggo               2336 non-null   object        
 7   floofer             2336 non-null   object        
 8   pupper              2336 non-null   object        
 9   puppo               2336 non-null   object        
 10  date                2336 non-null   datetime64[ns]
 11  time                2336 non-null   object        
 12  stage               2336 non-null   object        
dtypes: category(1), datetime64[ns](1), float64(2), o

### Code

In [206]:
# Drop columns 'doggo', 'floofer', 'pupper', and 'puppo'
df_archive_clean.drop(columns=['doggo', 'floofer', 'pupper', 'puppo','rating_denominator'], inplace=True)


### Test

In [207]:
df_archive_clean.sample(5)

,tweet_id,source,text,rating_numerator,name,date,time,stage
423,821765923262631936,Twitter for iPhone,This is Duchess. She uses dark doggo forces to levitate her toys. 13/10 magical af,13.0,Duchess,2017-01-18,17:07:18,doggo
791,773704687002451968,Twitter for iPhone,This is Loki. He knows he's adorable. One ear always pupared. 12/10 would snug in depicted fashion forever,12.0,Loki,2016-09-08,02:09:06,
1178,719551379208073216,Twitter for iPhone,This is Harnold. He accidentally opened the front facing camera. 10/10 get it together Harnold,10.0,Harnold,2016-04-11,15:43:12,
1091,737445876994609152,Twitter for iPhone,Just wanted to share this super rare Rainbow Floofer in case you guys haven't seen it yet. 13/10 colorful af,13.0,NaN,2016-05-31,00:49:32,floofer
2116,670427002554466305,Twitter for iPhone,This is a Deciduous Trimester mix named Spork. Only 1 ear works. No seat belt. Incredibly reckless. 9/10 still cute,9.0,Spork,2015-11-28,02:20:27,


In [208]:
df_archive_clean.info()
df_more_clean.info()
df_image_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2336 entries, 0 to 2355
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   tweet_id          2336 non-null   object        
 1   source            2336 non-null   category      
 2   text              2336 non-null   object        
 3   rating_numerator  2336 non-null   float64       
 4   name              1521 non-null   object        
 5   date              2336 non-null   datetime64[ns]
 6   time              2336 non-null   object        
 7   stage             2336 non-null   object        
dtypes: category(1), datetime64[ns](1), float64(1), object(5)
memory usage: 148.5+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2354 entries, 0 to 2353
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   tweet_id        2354 non-null   object
 1   retweet_count   2354 non-

## Issue
The three data table should be merge together into a single column

## Define
Merge all table on the tweet_id columns as primary key.

### Code


In [209]:
## merge archive and more on tweetid 

superdata = pd.merge(df_archive_clean, df_more_clean,  on = 'tweet_id', how = 'left')

In [210]:
super_data = pd.merge(superdata,df_image_clean, on = 'tweet_id', how = 'left')

In [211]:
superdata.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2336 entries, 0 to 2335
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   tweet_id          2336 non-null   object        
 1   source            2336 non-null   category      
 2   text              2336 non-null   object        
 3   rating_numerator  2336 non-null   float64       
 4   name              1521 non-null   object        
 5   date              2336 non-null   datetime64[ns]
 6   time              2336 non-null   object        
 7   stage             2336 non-null   object        
 8   retweet_count     2334 non-null   float64       
 9   favorite_count    2334 non-null   float64       
dtypes: category(1), datetime64[ns](1), float64(3), object(5)
memory usage: 185.0+ KB


<a id='Store'></a>

## Store Tables as CSV

In [212]:
df_archive_clean.to_csv('wrd_archive_clean_data.csv',index = False)
df_more_clean.to_csv('wrd_more_clean_tweet_archive.csv',index = False)
df_image_clean.to_csv('wrd_image_archive_clean_data.csv', index = False)
superdata.to_csv('wrd_merge_clean_data.csv', index = False)

In [213]:
superdata.head(10)

,tweet_id,source,text,rating_numerator,name,date,time,stage,retweet_count,favorite_count
0,892420643555336193,Twitter for iPhone,This is Phineas. He's a mystical boy. Only ever appears in the hole of a donut. 13/10,13.0,Phineas,2017-08-01,16:23:56,,8853.0,39467.0
1,892177421306343426,Twitter for iPhone,"This is Tilly. She's just checking pup on you. Hopes you're doing ok. If not, she's available for pats, snugs, boops, the whole bit. 13/10",13.0,Tilly,2017-08-01,00:17:27,,6514.0,33819.0
2,891815181378084864,Twitter for iPhone,This is Archie. He is a rare Norwegian Pouncing Corgo. Lives in the tall grass. You never know when one may strike. 12/10,12.0,Archie,2017-07-31,00:18:03,,4328.0,25461.0
3,891689557279858688,Twitter for iPhone,This is Darla. She commenced a snooze mid meal. 13/10 happens to the best of us,13.0,Darla,2017-07-30,15:58:51,,8964.0,42908.0
4,891327558926688256,Twitter for iPhone,"This is Franklin. He would like you to stop calling him ""cute."" He is a very fierce shark and should be respected as such. 12/10 #BarkWeek",12.0,Franklin,2017-07-29,16:00:24,,9774.0,41048.0
5,891087950875897856,Twitter for iPhone,Here we have a majestic great white breaching off South Africa's coast. Absolutely h*ckin breathtaking. 13/10 (IG: tucker_marlo) #BarkWeek,13.0,NaN,2017-07-29,00:08:17,,3261.0,20562.0
6,890971913173991426,Twitter for iPhone,Meet Jax. He enjoys ice cream so much he gets nervous around it. 13/10 help Jax enjoy more things by clicking below\n\n,13.0,Jax,2017-07-28,16:27:12,,2158.0,12041.0
7,890729181411237888,Twitter for iPhone,When you watch your owner call another dog a good boy but then they turn back to you and say you're a great boy. 13/10,13.0,NaN,2017-07-28,00:22:40,,16716.0,56848.0
8,890609185150312448,Twitter for iPhone,This is Zoey. She doesn't want to be one of the scary sharks. Just wants to be a snuggly pettable boatpet. 13/10 #BarkWeek,13.0,Zoey,2017-07-27,16:25:51,,4429.0,28226.0
9,890240255349198849,Twitter for iPhone,This is Cassie. She is a college pup. Studying international doggo communication and stick theory. 14/10 so elegant much sophisticate,14.0,Cassie,2017-07-26,15:59:51,doggo,7711.0,32467.0


## *All visualization and insight will be done on Tableau and will be published as dashboard*